In [1]:
import torch
import copy

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

import io
from utils import display_video

In [2]:
def WienerProcess(x0, dt=0.01, T=5):
    """
    Generate a Wiener process with n steps and T time units.
    """
    xt = copy.deepcopy(x0)
    xtraj = [copy.deepcopy(xt).unsqueeze(1)]
    for t in torch.linspace(0, 1, int(T/dt)):
        xt += torch.randn_like(x0) * np.sqrt(dt)
        xtraj.append(copy.deepcopy(xt.detach()).unsqueeze(1))
    return xt, torch.cat(xtraj, dim=1)

In [23]:
x0 = torch.zeros(500, 2)
T = 1
dt = 0.01
xT, xtraj = WienerProcess(x0, dt=dt, T=T)
xtraj.size()

torch.Size([500, 101, 2])

In [24]:
# Visualize the blobs
video = []
xtraj = xtraj.permute(1, 0, 2).detach().cpu()
for i, x in enumerate(xtraj):
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    ax.set_title("Brownian Motion" + f"\n (t={i*dt:.2f})")
    ax.set_aspect('equal')
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)
    ax.scatter(xtraj[i, :, 0], xtraj[i, :, 1], color='tab:blue', alpha=0.2, s=5)
    ax.scatter(xtraj[i, 0, 0], xtraj[i, 0, 1], color='r', s=10)
    
    # Save the figure to a buffer
    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight')  # Save to a buffer as a PNG image
    buf.seek(0)  # Rewind the buffer

    # Load the buffer into a PIL image and convert to NumPy array
    image = Image.open(buf)
    image_array = np.array(image)  # Convert to a NumPy array
    buf.close()  # Close the buffer
    plt.close()
    
    video.append(image_array)

In [25]:
display_video(video)
